# Taxi / Ride-Hailing Fleet Repositioning with Rolling-Horizon Convex Optimization (MPC)
Taxi/Ride-hailing systems must continuously decide where to position idle vehicles so that future trip requests can be served with low waiting times. When vehicles move without a passenger—often called deadheading (or empty miles) they incur operational cost without generating revenue, so good policies must balance service quality against empty repositioning cost.

In this notebook, we study dynamic fleet rebalancing using public trip records from the NYC Taxi & Limousine Commission (TLC). The TLC yellow-taxi data contains pickup/dropoff timestamps and pickup/dropoff zone identifiers, enabling us to aggregate origin–destination demand over time and space.

We formulate this rebalancing as a convex, multi-period flow optimization solved in a receding-horizon (Model Predictive Control, MPC) loop: at each time step we 

1. State: idle vehicles per zone at the current time.
2. Forecast: expected demand for the next few hours (simple time-of-week averages from training data).
3. Optimize (convex): solve a multi-period, time-expanded flow model that decides:
   - how many trips to serve in each zone pair, and
   - how many vehicles to move empty between zones (reposition / deadhead flows),
   trading off customer service vs. deadheading cost.
4. Apply only the first step of the plan.
5. Simulate reality: reveal the actual demand from held-out TLC data, update vehicle locations with travel-time lags, and repeat.

---

Data:
- NYC TLC Trip Record Data (monthly Parquet download links)
- Taxi zone lookup table (`LocationID → Borough/Zone/Service zone`)
- Yellow taxi data dictionary for column meanings


## Problem at a glance

- City is partitioned into zones $i \in \{1,\dots,K\}$.
- Time is discretized into steps $t = 0,1,\dots$ (e.g., 15 minutes).
- State $s_{i,t}$: idle vehicles in zone $i$ at step $t$.
- Demand $d_{ij,t}$: ride requests from zone $i$ to $j$ at step $t$.
- Decisions:
  - $x_{ij,t}$: rides served from $i \to j$ (bounded by demand)
  - $r_{ij,t}$: empty reposition moves from $i \to j$

We solve a multi-period, time-expanded network-flow problem and run it in a rolling horizon loop.

## Mathematical formulation

Let $H$ be the planning horizon (number of future time steps optimized each time).

Decision variables (continuous):
- $x_{ij,t} \ge 0$ served demand (rides) for $t=0,\dots,H-1$
- $r_{ij,t} \ge 0$ reposition (deadhead) flows
- $u_{ij,t} \ge 0$ unmet demand slack
- $s_{i,t} \ge 0$ idle vehicles for $t=0,\dots,H$

Constraints
1. Demand bounds $0 \le x_{ij,t} \le \hat d_{ij,t}$
and we set $u_{ij,t} = \hat d_{ij,t} - x_{ij,t}$.

2. Idle vehicle availability
$$
\sum_j x_{ij,t} + \sum_j r_{ij,t} \le s_{i,t}
$$

3. Fleet dynamics with travel-time lags

We estimate an integer travel-time lag $\tau_{ij}\in\{1,\dots,\tau_{\max}\}$ (in steps) from data.

Vehicles that depart at time $t$ from $i\to j$ arrive at $j$ at time $t+\tau_{ij}$. Therefore, for each zone $i$ and time $t$:

$$
s_{i,t+1} = s_{i,t}
- \sum_j (x_{ij,t} + r_{ij,t})
+ \sum_k \Big( x_{k i, t-\tau_{ki}+1} + r_{k i, t-\tau_{ki}+1} \Big)
$$
where terms with negative indices are treated as 0.

Objective (LP)
$$
\min \sum_{t,i,j} c^{repo}_{ij}\, r_{ij,t} \;+\; \lambda \sum_{t,i,j} u_{ij,t}
$$

- $c^{repo}_{ij}$: deadhead cost (we use median travel time from data as a proxy)
- $\lambda$: penalty for unmet demand (service-level priority)

Optional convex smoothing (QP):
Add $\eta \sum_t \|s_{\cdot,t+1}-s_{\cdot,t}\|_2^2$ to reduce oscillatory repositioning.

In [7]:
#!pip install pyarrow

In [1]:

import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pyarrow 
import cvxpy as cp

SEED = 7
rng = np.random.default_rng(SEED)

print("cvxpy installed solvers:", cp.installed_solvers())

c:\Users\salmank\anaconda3\envs\pymc_env_5\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cvxpy installed solvers: ['CLARABEL', 'CVXOPT', 'ECOS', 'ECOS_BB', 'GLOP', 'GLPK', 'GLPK_MI', 'OSQP', 'PDLP', 'SCIPY', 'SCS']


#### Download NYC TLC data (Parquet) + zone lookup (CSV)

In [2]:

import urllib.request

DATA_DIR = "data_tlc"
os.makedirs(DATA_DIR, exist_ok=True)

# Choose a month. You can change this to any month available on the TLC page.
MONTH = "2024-01"

YELLOW_URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{MONTH}.parquet"
ZONE_URL   = "https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv"

YELLOW_PATH = os.path.join(DATA_DIR, f"yellow_tripdata_{MONTH}.parquet")
ZONE_PATH   = os.path.join(DATA_DIR, "taxi_zone_lookup.csv")

DOWNLOAD = True

def download(url: str, path: str):
    if os.path.exists(path):
        print(f"✅ Exists: {path}")
        return
    print(f"⬇️ Downloading: {url}")
    urllib.request.urlretrieve(url, path)
    print(f"✅ Saved to: {path}")

if DOWNLOAD:
    download(ZONE_URL, ZONE_PATH)
    download(YELLOW_URL, YELLOW_PATH)

✅ Exists: data_tlc\taxi_zone_lookup.csv
✅ Exists: data_tlc\yellow_tripdata_2024-01.parquet


#### Load and clean the data

We only need:
- pickup datetime (`tpep_pickup_datetime`)
- dropoff datetime (`tpep_dropoff_datetime`)
- pickup zone ID (`PULocationID`)
- dropoff zone ID (`DOLocationID`)

We filter to a manageable time window and optionally sample rows.

In [14]:
# Column subset (keeps memory reasonable)
COLS = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "PULocationID", "DOLocationID"]

# Optional row sampling after load (set to None for full month)
N_SAMPLE = 3_000_000  # adjust based on your machine

df = pd.read_parquet(YELLOW_PATH, columns=COLS,  engine="pyarrow")

# Basic cleaning
df = df.dropna(subset=["PULocationID", "DOLocationID", "tpep_pickup_datetime", "tpep_dropoff_datetime"]).copy()


# Ensure integer zone IDs
df["PULocationID"] = df["PULocationID"].astype(int)
df["DOLocationID"] = df["DOLocationID"].astype(int)

pickup  = pd.to_datetime(df["tpep_pickup_datetime"])
dropoff = pd.to_datetime(df["tpep_dropoff_datetime"])
df["duration_min"] = (dropoff - pickup).dt.total_seconds() / 60.0
df['pickup_dt'] = pickup

df.head()


,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,duration_min,pickup_dt
0,2024-01-01 00:57:55,2024-01-01 01:17:43,186,79,19.800000,2024-01-01 00:57:55
1,2024-01-01 00:03:00,2024-01-01 00:09:36,140,236,6.600000,2024-01-01 00:03:00
2,2024-01-01 00:17:06,2024-01-01 00:35:01,236,79,17.916667,2024-01-01 00:17:06
3,2024-01-01 00:36:38,2024-01-01 00:44:56,79,211,8.300000,2024-01-01 00:36:38
4,2024-01-01 00:46:51,2024-01-01 00:52:57,211,148,6.100000,2024-01-01 00:46:51


In [12]:
zones = pd.read_csv(ZONE_PATH)
zones

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone
...,...,...,...,...
260,261,Manhattan,World Trade Center,Yellow Zone
261,262,Manhattan,Yorkville East,Yellow Zone
262,263,Manhattan,Yorkville West,Yellow Zone
263,264,Unknown,NaN,NaN


#### Build demand tensors and travel-time estimates

We choose:
- time step length `DT_MIN` (e.g., 15 minutes)
- a subset of zones (`K_ZONES`) to keep optimization fast
- a train/test split within the month:
  - train: compute demand profiles and travel-time medians
  - test: backtest the MPC policy

In [15]:
# train/test split
month_start = df["pickup_dt"].min().normalize()
train_start = month_start
train_end   = train_start + pd.Timedelta(days=21)
test_start  = train_end
test_end    = test_start + pd.Timedelta(days=7)

df_train = df[(df["pickup_dt"] >= train_start) & (df["pickup_dt"] < train_end)].copy()
df_test  = df[(df["pickup_dt"] >= test_start)  & (df["pickup_dt"] < test_end)].copy()

print("Train window:", train_start, "to", train_end, "rows:", len(df_train))
print("Test  window:", test_start,  "to", test_end,  "rows:", len(df_test))


Train window: 2002-12-31 00:00:00 to 2003-01-21 00:00:00 rows: 2
Test  window: 2003-01-21 00:00:00 to 2003-01-28 00:00:00 rows: 0


In [21]:

DT_MIN = 15
K_ZONES = 100              # keep moderate; increase if you want
H = 12                    # horizon length in steps (12*15min = 3 hours)
TAU_MAX = 8               # max travel-time lag in steps (8*15min = 2 hours)

top_zones = df_train["PULocationID"].value_counts().head(K_ZONES).index.to_numpy()
zone_to_idx = {z:i for i,z in enumerate(top_zones)}
idx_to_zone = {i:z for z,i in zone_to_idx.items()}

print("Example zones:", top_zones[:10])

Example zones: [170]


In [22]:
def discretize_time(ts: pd.Series, dt_min: int) -> pd.Series:
    return ts.dt.floor(f"{dt_min}min")

df_train_k = df_train[df_train["PULocationID"].isin(top_zones) & df_train["DOLocationID"].isin(top_zones)].copy()
df_test_k  = df_test[df_test["PULocationID"].isin(top_zones) & df_test["DOLocationID"].isin(top_zones)].copy()

df_train_k["tbin"] = discretize_time(df_train_k["pickup_dt"], DT_MIN)
df_test_k["tbin"]  = discretize_time(df_test_k["pickup_dt"], DT_MIN)

df_train_k


,tpep_pickup_datetime,tpep_dropoff_datetime,PULocationID,DOLocationID,duration_min,pickup_dt,tbin
53119,2002-12-31 22:59:39,2002-12-31 23:05:41,170,170,6.033333,2002-12-31 22:59:39,2002-12-31 22:45:00
53120,2002-12-31 22:59:39,2002-12-31 23:05:41,170,170,6.033333,2002-12-31 22:59:39,2002-12-31 22:45:00


In [ ]:

t_index = pd.date_range(df_test_k["tbin"].min(), df_test_k["tbin"].max(), freq=f"{DT_MIN}min")
t_to_idx = {t:i for i,t in enumerate(t_index)}

T_test = len(t_index)
K = len(top_zones)

d_real = np.zeros((T_test, K, K), dtype=np.float32)

for row in df_test_k.itertuples(index=False):
    t = row.tbin
    if t not in t_to_idx:
        continue
    ti = t_to_idx[t]
    i = zone_to_idx[row.PULocationID]
    j = zone_to_idx[row.DOLocationID]
    d_real[ti, i, j] += 1.0

print("Test tensor shape:", d_real.shape, "nonzeros:", np.count_nonzero(d_real))

In [ ]:

od_med = (df_train_k
          .groupby(["PULocationID", "DOLocationID"])["duration_min"]
          .median()
          .reset_index())

global_med = float(df_train_k["duration_min"].median())
tt_min = np.full((K, K), global_med, dtype=np.float32)

for row in od_med.itertuples(index=False):
    i = zone_to_idx[row.PULocationID]
    j = zone_to_idx[row.DOLocationID]
    tt_min[i, j] = float(row.duration_min)

tau = np.clip(np.ceil(tt_min / DT_MIN).astype(int), 1, TAU_MAX)
c_repo = tt_min.copy()

print("Median duration (min) global:", global_med)
print("tau stats (steps): min", tau.min(), "max", tau.max())

## 4) Demand forecasting baseline (time-of-week average)

In [ ]:

steps_per_day = int(24*60/DT_MIN)

df_train_k["weekday"] = df_train_k["tbin"].dt.weekday
df_train_k["step_of_day"] = (df_train_k["tbin"].dt.hour * 60 + df_train_k["tbin"].dt.minute) // DT_MIN
df_train_k["date"] = df_train_k["tbin"].dt.date

weekday_days = df_train_k.groupby("weekday")["date"].nunique().to_dict()

grp = (df_train_k
       .groupby(["weekday","step_of_day","PULocationID","DOLocationID"])
       .size()
       .reset_index(name="cnt"))

avg = np.zeros((7, steps_per_day, K, K), dtype=np.float32)

for row in grp.itertuples(index=False):
    w, s = int(row.weekday), int(row.step_of_day)
    i = zone_to_idx[row.PULocationID]
    j = zone_to_idx[row.DOLocationID]
    denom = max(1, weekday_days.get(w, 1))
    avg[w, s, i, j] = float(row.cnt) / denom

def forecast_demand(t0: pd.Timestamp, H: int) -> np.ndarray:
    out = np.zeros((H, K, K), dtype=np.float32)
    for h in range(H):
        th = t0 + pd.Timedelta(minutes=DT_MIN*h)
        w = th.weekday()
        s = (th.hour*60 + th.minute)//DT_MIN
        out[h] = avg[w, s]
    return out

print("avg tensor shape:", avg.shape)

## 5) Convex MPC optimizer (CVXPY)

In [ ]:

def solve_mpc_lp(
    s0: np.ndarray,
    d_hat: np.ndarray,     # (H,K,K)
    c_repo: np.ndarray,    # (K,K)
    tau: np.ndarray,       # (K,K) integer steps
    lambda_unmet: float = 5.0,
    eta_smooth: float = 0.0,
    solver: str | None = None,
):
    H, K, _ = d_hat.shape
    s0 = s0.astype(float)

    x = cp.Variable((H, K, K), nonneg=True)
    r = cp.Variable((H, K, K), nonneg=True)
    s = cp.Variable((H+1, K), nonneg=True)
    u = cp.Variable((H, K, K), nonneg=True)

    cons = [s[0, :] == s0, x <= d_hat, u == d_hat - x]

    for t in range(H):
        cons += [cp.sum(x[t, :, :], axis=1) + cp.sum(r[t, :, :], axis=1) <= s[t, :]]

    for t in range(H):
        departures = cp.sum(x[t, :, :], axis=1) + cp.sum(r[t, :, :], axis=1)

        arrivals_expr = []
        for i in range(K):
            incoming_terms = []
            for k in range(K):
                lag = int(tau[k, i])
                dep_t = t - lag + 1
                if 0 <= dep_t < H:
                    incoming_terms.append(x[dep_t, k, i] + r[dep_t, k, i])
            arrivals_expr.append(cp.sum(cp.hstack(incoming_terms)) if incoming_terms else 0.0)

        arrivals = cp.hstack(arrivals_expr)
        cons += [s[t+1, :] == s[t, :] - departures + arrivals]

    obj = cp.sum(cp.multiply(c_repo[None, :, :], r)) + lambda_unmet * cp.sum(u)
    if eta_smooth > 0:
        obj += eta_smooth * cp.sum_squares(s[1:, :] - s[:-1, :])

    prob = cp.Problem(cp.Minimize(obj), cons)

    if solver is None:
        for cand in ["ECOS", "OSQP", "CLARABEL", "SCS"]:
            if cand in cp.installed_solvers():
                solver = cand
                break

    prob.solve(solver=solver, verbose=False)
    if prob.status not in ("optimal", "optimal_inaccurate"):
        raise RuntimeError(f"MPC solve failed: status={prob.status}")

    x0 = np.maximum(x.value[0], 0)
    r0 = np.maximum(r.value[0], 0)
    return x0, r0, float(prob.value), prob.status, solver

## 6) Simulator + backtesting loop

In [ ]:

from dataclasses import dataclass

@dataclass
class SimResult:
    service_rate: np.ndarray
    unmet: np.ndarray
    deadhead_cost: np.ndarray
    idle_total: np.ndarray
    solver_time: np.ndarray
    x_served_total: float
    demand_total: float
    deadhead_total: float

def run_backtest(policy: str, H_policy: int, lambda_unmet=5.0, eta_smooth=0.0, fleet_size=4000):
    pickup_counts = df_train_k["PULocationID"].value_counts().reindex(top_zones).fillna(0).to_numpy(dtype=float)
    p = pickup_counts / max(1.0, pickup_counts.sum())
    s_idle = np.round(fleet_size * p).astype(float)

    transit = [np.zeros(K, dtype=float) for _ in range(TAU_MAX)]

    service_rate = np.zeros(T_test, dtype=float)
    unmet = np.zeros(T_test, dtype=float)
    deadhead_cost = np.zeros(T_test, dtype=float)
    idle_total = np.zeros(T_test, dtype=float)
    solver_time = np.zeros(T_test, dtype=float)

    x_served_total = 0.0
    demand_total = float(d_real.sum())
    deadhead_total = 0.0

    for t in tqdm(range(T_test), desc=f"Backtest: {policy}"):
        arrivals_now = transit.pop(0)
        transit.append(np.zeros(K, dtype=float))
        s_idle += arrivals_now

        idle_total[t] = s_idle.sum()
        d_t = d_real[t].astype(float)

        if policy == "none":
            x_plan = d_t.copy()
            r_plan = np.zeros((K, K), dtype=float)
        else:
            t0 = t_index[t]
            d_hat = forecast_demand(t0, H_policy)

            import time
            t_start = time.time()
            x0, r0, _, _, _ = solve_mpc_lp(
                s0=s_idle,
                d_hat=d_hat,
                c_repo=c_repo,
                tau=tau,
                lambda_unmet=lambda_unmet,
                eta_smooth=eta_smooth,
            )
            solver_time[t] = time.time() - t_start
            x_plan, r_plan = x0, r0

        x_serv = np.minimum(x_plan, d_t)

        for i in range(K):
            avail = s_idle[i]
            serve_out = x_serv[i, :].sum()
            if serve_out > avail + 1e-9:
                x_serv[i, :] *= (avail / serve_out)
                serve_out = avail

            avail_left = avail - serve_out
            repo_out = r_plan[i, :].sum()
            if repo_out > avail_left + 1e-9 and repo_out > 0:
                r_plan[i, :] *= (avail_left / repo_out)

        dep = x_serv.sum(axis=1) + r_plan.sum(axis=1)
        s_idle -= dep
        s_idle = np.maximum(s_idle, 0.0)

        for i in range(K):
            for j in range(K):
                flow = x_serv[i, j] + r_plan[i, j]
                if flow <= 0:
                    continue
                lag = int(tau[i, j])
                lag = max(1, min(TAU_MAX, lag))
                transit[lag-1][j] += flow

        served = float(x_serv.sum())
        unmet_t = float(d_t.sum() - served)
        cost_t = float((c_repo * r_plan).sum())

        x_served_total += served
        deadhead_total += cost_t

        service_rate[t] = served / (float(d_t.sum()) + 1e-9)
        unmet[t] = unmet_t
        deadhead_cost[t] = cost_t

    return SimResult(
        service_rate=service_rate,
        unmet=unmet,
        deadhead_cost=deadhead_cost,
        idle_total=idle_total,
        solver_time=solver_time,
        x_served_total=x_served_total,
        demand_total=demand_total,
        deadhead_total=deadhead_total,
    )

In [ ]:

# Run policies
res_none = run_backtest("none",   H_policy=1,  fleet_size=4000)
res_myo  = run_backtest("myopic", H_policy=1,  fleet_size=4000, lambda_unmet=8.0)
res_mpc  = run_backtest("mpc",    H_policy=H,  fleet_size=4000, lambda_unmet=8.0, eta_smooth=1e-3)

def summarize(name, res: SimResult):
    return {
        "Policy": name,
        "Total demand": res.demand_total,
        "Total served": res.x_served_total,
        "Service rate": res.x_served_total / max(1e-9, res.demand_total),
        "Total deadhead cost": res.deadhead_total,
        "Avg solver time (s)": float(np.mean(res.solver_time[res.solver_time>0])) if np.any(res.solver_time>0) else 0.0,
    }

summary = pd.DataFrame([
    summarize("No reposition", res_none),
    summarize("Myopic H=1",    res_myo),
    summarize(f"MPC H={H}",    res_mpc),
])

summary

In [ ]:

plt.figure(figsize=(12,4))
plt.plot(res_none.service_rate, label="No reposition")
plt.plot(res_myo.service_rate,  label="Myopic H=1")
plt.plot(res_mpc.service_rate,  label=f"MPC H={H}")
plt.xlabel("Time step (15-min)")
plt.ylabel("Service rate (served / demand)")
plt.title("Service rate over time")
plt.legend()
plt.show()

In [ ]:

plt.figure(figsize=(12,4))
plt.plot(np.cumsum(res_none.deadhead_cost), label="No reposition")
plt.plot(np.cumsum(res_myo.deadhead_cost),  label="Myopic H=1")
plt.plot(np.cumsum(res_mpc.deadhead_cost),  label=f"MPC H={H}")
plt.xlabel("Time step (15-min)")
plt.ylabel("Cumulative deadhead cost (minutes)")
plt.title("Cumulative deadhead cost over time")
plt.legend()
plt.show()

## 7) Stress testing scenarios

Because we have a simulator, you can create "what-if" scenarios without changing the optimizer:
- Demand shock: multiply demand in a set of zones for a period (event letting out).
- Congestion shock: increase travel times $tt_{ij}$ (and thus $\tau_{ij}$, $c^{repo}_{ij}$) for certain OD pairs.
- Fleet shortage: reduce `fleet_size`.

Below is a simple demand shock example.

In [ ]:

def apply_demand_shock(d_real_base: np.ndarray, zone_subset_idx, t_start, t_end, factor=1.5):
    d = d_real_base.copy()
    d[t_start:t_end, zone_subset_idx, :] *= factor
    return d

shock_zones = list(range(min(5, K)))
t_start_shock, t_end_shock = 24, 36  # adjust based on your test start time

d_backup = d_real.copy()
d_real = apply_demand_shock(d_real, shock_zones, t_start_shock, t_end_shock, factor=1.8)

res_mpc_shock = run_backtest("mpc", H_policy=H, fleet_size=4000, lambda_unmet=8.0, eta_smooth=1e-3)

d_real = d_backup

print("Base MPC service rate:", res_mpc.x_served_total / res_mpc.demand_total)
print("Shock MPC service rate:", res_mpc_shock.x_served_total / res_mpc_shock.demand_total)

In [ ]:

plt.figure(figsize=(12,4))
plt.plot(res_mpc.service_rate, label="MPC (base)")
plt.plot(res_mpc_shock.service_rate, label="MPC (demand shock)")
plt.axvspan(t_start_shock, t_end_shock, alpha=0.2, label="Shock window")
plt.xlabel("Time step")
plt.ylabel("Service rate")
plt.title("MPC robustness under a demand shock")
plt.legend()
plt.show()

## Next upgrades (optional)

- Add fairness constraints (minimum service per borough / zone group)
- Use HVFHV trip records for a closer ride-hailing proxy (also on the TLC page)
- Use the taxi zone shapefile to plot maps (geopandas)